In [1]:
## Standard libraries
import os
from copy import deepcopy

## Imports for plotting
import matplotlib.pyplot as plt
plt.set_cmap('cividis')
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.set()
import time

## tqdm for loading bars
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

## Torchvision
import torchvision
from torchvision.datasets import STL10
from torchvision import transforms

# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Import tensorboard
%load_ext tensorboard

NUM_WORKERS = os.cpu_count()

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)
print("Number of workers:", NUM_WORKERS)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  # Remove the CWD from sys.path while we load stuff.


Device: cuda:0
Number of workers: 2


In [2]:
import urllib.request
from urllib.error import HTTPError
from torchvision import transforms
import os


In [3]:
class ContrastiveTransformations(object):
    
    def __init__(self, base_transforms, n_views=2):
        self.base_transforms = base_transforms
        self.n_views = n_views
        
    def __call__(self, x):
        return [self.base_transforms(x) for i in range(self.n_views)]

In [4]:
contrast_transforms = transforms.Compose([transforms.RandomHorizontalFlip(),
                                          transforms.RandomResizedCrop(size=96),
                                          transforms.RandomApply([
                                              transforms.ColorJitter(brightness=0.5, 
                                                                     contrast=0.5, 
                                                                     saturation=0.5, 
                                                                     hue=0.1)
                                          ], p=0.8),
                                          transforms.RandomGrayscale(p=0.2),
                                          transforms.GaussianBlur(kernel_size=9),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.5,), (0.5,))
                                         ])

In [5]:
import pandas as pd
df = pd.read_csv("../input/sdss-images/data.csv")
df.iloc[0]["image"]

'image_0000.jpg'

In [6]:
df['class'].value_counts()

GALAXY    4998
STAR      4152
QSO        850
Name: class, dtype: int64

In [7]:
import numpy as np
import torch.nn.functional as nnf
from torch.utils.data.dataset import Dataset
from skimage import io as skio
from skimage import transform
from torchvision import transforms as T

class SDSSDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        self.df = pd.read_csv(csv_file)
        label_dict = {'GALAXY':0,'STAR':1,'QSO':2}
        self.df['class'] = self.df['class'].apply(lambda x : label_dict[x])
        print(self.df.head(2))
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,self.df.iloc[idx]["image"])
        image = skio.imread(img_name).reshape(3,128,128).astype(np.float32)
        torch_tensor = torch.from_numpy(image).unsqueeze(0)
        out = nnf.interpolate(torch_tensor, size=(96, 96), mode='nearest').squeeze(0)
        image = out.cpu().detach().numpy().reshape(3,96,96)
        label = self.df.iloc[idx]["class"]
        return image,label

In [8]:
import shutil
import pandas as pd
balanced_set = pd.read_csv("../input/sdss-images/equal_splits.csv")
print(balanced_set.columns)
balanced_images = list(balanced_set[balanced_set.columns[1]])
balanced_images.extend(list(balanced_set[balanced_set.columns[2]]))
balanced_images.extend(list(balanced_set[balanced_set.columns[3]]))

Index(['Unnamed: 0', 'trimmed traininng  qalaxy', 'trimmed traininng  qsd',
       'trimmed traininng  star'],
      dtype='object')


In [9]:
df = pd.read_csv("../input/sdss-images/data.csv")
balanced_images = df['image']

In [10]:
!rm -rf balanced_images/*
!mkdir balanced_images

In [11]:
for img in balanced_images:
    shutil.copy("../input/sdss-images/images (1)/images/"+img,"./balanced_images/"+img)

In [12]:
balanced_df = df.loc[df['image'].isin(balanced_images)].reset_index().drop(['index'],axis=1)

In [13]:
from sklearn.model_selection import train_test_split
balanced_train,balanced_test = train_test_split(balanced_df,test_size=0.2,stratify=balanced_df['class'])
balanced_train.to_csv("balanced_train.csv",index=False),balanced_test.to_csv("balanced_test.csv",index=False)

(None, None)

In [14]:
class SimCLR(pl.LightningModule):
    
    def __init__(self, hidden_dim, lr, temperature, weight_decay, max_epochs=500):
        super().__init__()
        self.save_hyperparameters()
        assert self.hparams.temperature > 0.0, 'The temperature must be a positive float!'
        # Base model f(.)
        self.convnet = torchvision.models.resnet18(num_classes=4*hidden_dim)  # Output of last linear layer
        # The MLP for g(.) consists of Linear->ReLU->Linear 
        self.convnet.fc = nn.Sequential(
            self.convnet.fc,  # Linear(ResNet output, 4*hidden_dim)
            nn.ReLU(inplace=True),
            nn.Linear(4*hidden_dim, hidden_dim)
        )

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), 
                                lr=self.hparams.lr, 
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                            T_max=self.hparams.max_epochs,
                                                            eta_min=self.hparams.lr/50)
        return [optimizer], [lr_scheduler]
        
    def info_nce_loss(self, batch, mode='train'):
        imgs, _ = batch
        imgs = list([i.cpu().detach().numpy() for i in imgs])
        imgs = torch.tensor(imgs).to(device)
        # Encode all images
        feats = self.convnet(imgs.float())
        # Calculate cosine similarity
        cos_sim = F.cosine_similarity(feats[:,None,:], feats[None,:,:], dim=-1)
        # Mask out cosine similarity to itself
        self_mask = torch.eye(cos_sim.shape[0], dtype=torch.bool, device=cos_sim.device)
        cos_sim.masked_fill_(self_mask, -9e15)
        # Find positive example -> batch_size//2 away from the original example
        pos_mask = self_mask.roll(shifts=cos_sim.shape[0]//2, dims=0)
        # InfoNCE loss
        cos_sim = cos_sim / self.hparams.temperature
        nll = -cos_sim[pos_mask] + torch.logsumexp(cos_sim, dim=-1)
        nll = nll.mean()
        
        # Logging loss
        self.log(mode+'_loss', nll)
        # Get ranking position of positive example
        comb_sim = torch.cat([cos_sim[pos_mask][:,None],  # First position positive example
                              cos_sim.masked_fill(pos_mask, -9e15)], 
                             dim=-1)
        sim_argsort = comb_sim.argsort(dim=-1, descending=True).argmin(dim=-1)
        # Logging ranking metrics
        self.log(mode+'_acc_top1', (sim_argsort == 0).float().mean())
        self.log(mode+'_acc_top5', (sim_argsort < 5).float().mean())
        self.log(mode+'_acc_mean_pos', 1+sim_argsort.float().mean())
        
        return nll
        
    def training_step(self, batch, batch_idx):
        
        return self.info_nce_loss(batch, mode='train')
        
    def validation_step(self, batch, batch_idx):
        self.info_nce_loss(batch, mode='val')
        

In [15]:
model = SimCLR(hidden_dim=128,lr=5e-4,temperature=0.07,weight_decay=1e-4,max_epochs=50)
model.load_state_dict(torch.load("../input/simclr-ml701/simclr_resnet.pth"))

<All keys matched successfully>

In [16]:
def train_simclr(batch_size, max_epochs=500, **kwargs):
    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, 'SimCLR'),
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=max_epochs,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode='max', monitor='val_acc_top5'),
                                    LearningRateMonitor('epoch')])
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, 'SimCLR.ckpt')
    if os.path.isfile(pretrained_filename):
        print(f'Found pretrained model at {pretrained_filename}, loading...')
        model = SimCLR.load_from_checkpoint(pretrained_filename) # Automatically loads the model with the saved hyperparameters
    else:
        train_loader = data.DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                                       drop_last=True, pin_memory=True, num_workers=NUM_WORKERS)
        val_loader = data.DataLoader(dataset, batch_size=batch_size, shuffle=False, 
                                     drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
        pl.seed_everything(42) # To be reproducable
        model = SimCLR(max_epochs=max_epochs, **kwargs)
        trainer.fit(model, train_loader, val_loader)
        model = SimCLR.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    return model

In [17]:
class LogisticRegression(pl.LightningModule):
    
    def __init__(self, feature_dim, num_classes, lr, weight_decay, max_epochs=100):
        super().__init__()
        self.save_hyperparameters()
        # Mapping from representation h to classes
        self.model = nn.Linear(feature_dim, num_classes)

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), 
                                lr=self.hparams.lr, 
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, 
                                                      milestones=[int(self.hparams.max_epochs*0.6), 
                                                                  int(self.hparams.max_epochs*0.8)], 
                                                      gamma=0.1)
        return [optimizer], [lr_scheduler]
        
    def _calculate_loss(self, batch, mode='train'):
        feats, labels = batch
        preds = self.model(feats)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        self.log(mode + '_loss', loss)
        self.log(mode + '_acc', acc)
        return loss        
        
    def training_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode='train')
        
    def validation_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode='val')
        
    def test_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode='test')
    
    def forward(self,batch):
        return self.model(batch)

In [18]:
img_transforms = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5,), (0.5,))])
train_img_data = SDSSDataset("./balanced_train.csv","./balanced_images")
test_img_data = SDSSDataset("./balanced_test.csv","./balanced_images")
print("Number of training examples:", len(train_img_data))
print("Number of test examples:", len(test_img_data))

   class           image
0      1  image_3468.jpg
1      1  image_7907.jpg
   class           image
0      2  image_0106.jpg
1      0  image_0440.jpg
Number of training examples: 8000
Number of test examples: 2000


<!-- Next, we implement a small function to encode all images in our datasets. The output representations are then used as inputs to the Logistic Regression model. -->

In [19]:
@torch.no_grad()
def prepare_data_features(model, dataset):
    # Prepare model
    network = deepcopy(model.convnet)
    network.fc = nn.Identity()  # Removing projection head g(.)
    network.eval()
    network.to(device)
    
    # Encode all images
    data_loader = data.DataLoader(dataset, batch_size=64, num_workers=NUM_WORKERS, shuffle=False, drop_last=False)
    feats, labels = [], []
    for batch_imgs, batch_labels in tqdm(data_loader):
        batch_imgs = batch_imgs.to(device)
        batch_feats = network(batch_imgs)
        feats.append(batch_feats.detach().cpu())
        labels.append(batch_labels)
    
    feats = torch.cat(feats, dim=0)
    labels = torch.cat(labels, dim=0)
    
    # Sort images by labels
    labels, idxs = labels.sort()
    feats = feats[idxs]
    
    return data.TensorDataset(feats, labels)

In [20]:
train_feats_simclr = prepare_data_features(model, train_img_data)
test_feats_simclr = prepare_data_features(model, test_img_data)

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [25]:
def train_logreg(batch_size, train_feats_data, test_feats_data, model_suffix, max_epochs=100, **kwargs):
#     default_root_dir=os.path.join(CHECKPOINT_PATH, "LogisticRegression"),
    trainer = pl.Trainer(
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=max_epochs,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode='max', monitor='val_acc'),
                                    LearningRateMonitor("epoch")],
                         enable_progress_bar=False,
                         check_val_every_n_epoch=10)
    trainer.logger._default_hp_metric = None
    # Data loaders
    train_loader = data.DataLoader(train_feats_data, batch_size=batch_size, shuffle=True, 
                                   drop_last=False, pin_memory=True, num_workers=0)
    test_loader = data.DataLoader(test_feats_data, batch_size=batch_size, shuffle=False, 
                                  drop_last=False, pin_memory=True, num_workers=0)

    # Check whether pretrained model exists. If yes, load it and skip training
#     pretrained_filename = os.path.join(CHECKPOINT_PATH, f"LogisticRegression_{model_suffix}.ckpt")
#     if os.path.isfile(pretrained_filename):
#         print(f"Found pretrained model at {pretrained_filename}, loading...")
#         model = LogisticRegression.load_from_checkpoint(pretrained_filename)
#     else:
    pl.seed_everything(42)  # To be reproducable
    model = LogisticRegression(**kwargs)
    trainer.fit(model, train_loader, test_loader)
    model = LogisticRegression.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    return trainer,model#, result

In [26]:
results = {}
trainer,model = train_logreg(batch_size=64,
                                    train_feats_data=train_feats_simclr,
                                    test_feats_data=test_feats_simclr,
                                    model_suffix="any",
                                    feature_dim=train_feats_simclr.tensors[0].shape[1],
                                    num_classes=3,
                                    lr=1e-3,
                                    weight_decay=1e-3)
    

In [27]:
test_loader = data.DataLoader(test_feats_simclr, batch_size=1, shuffle=False, 
                                  drop_last=False, pin_memory=True, num_workers=0)

In [28]:
preds = list()
labels = list()
for i in test_loader:
    preds.append(model.forward(i[0]))
    labels.append(i[1])

In [29]:
import numpy as np
pred_argmax = [np.argmax(list(p.detach().cpu().numpy()[0])) for p in preds]
labels_argmax = [l.detach().cpu().numpy()[0] for l in labels]

In [30]:
from sklearn.metrics import classification_report
classification_report(pred_argmax,labels_argmax,digits=6).split("\n")

['              precision    recall  f1-score   support',
 '',
 '           0   0.954000  0.946429  0.950199      1008',
 '           1   0.925301  0.883774  0.904061       869',
 '           2   0.535294  0.739837  0.621160       123',
 '',
 '    accuracy                       0.906500      2000',
 '   macro avg   0.804865  0.856680  0.825140      2000',
 'weighted avg   0.915780  0.906500  0.909916      2000',
 '']

# Resnet fine tune

In [31]:
def get_net(model):
    net = model
#     net = torchvision.models.resnet18(pretrained=True)
    
    # Substitute the FC output layer
    net.fc = torch.nn.Linear(net.fc.in_features, 10)
    torch.nn.init.xavier_uniform_(net.fc.weight)
    return net

def train(net, train_dataloader, valid_dataloader, criterion, optimizer, scheduler=None, epochs=10, device='cpu', checkpoint_epochs=10):
    start = time.time()
    print(f'Training for {epochs} epochs on {device}')
    
    for epoch in range(1,epochs+1):
        print(f"Epoch {epoch}/{epochs}")
        
        net.train()  # put network in train mode for Dropout and Batch Normalization
        train_loss = torch.tensor(0., device=device)  # loss and accuracy tensors are on the GPU to avoid data transfers
        train_accuracy = torch.tensor(0., device=device)
        for X, y in train_dataloader:
            X = X.to(device)
            y = y.to(device)
            preds = net(X)
            loss = criterion(preds, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                train_loss += loss * train_dataloader.batch_size
                train_accuracy += (torch.argmax(preds, dim=1) == y).sum()
        
        if valid_dataloader is not None:
            net.eval()  # put network in train mode for Dropout and Batch Normalization
            valid_loss = torch.tensor(0., device=device)
            valid_accuracy = torch.tensor(0., device=device)
            with torch.no_grad():
                for X, y in valid_dataloader:
                    X = X.to(device)
                    y = y.to(device)
                    preds = net(X)
                    loss = criterion(preds, y)

                    valid_loss += loss * valid_dataloader.batch_size
                    valid_accuracy += (torch.argmax(preds, dim=1) == y).sum()
        
        if scheduler is not None: 
            scheduler.step()
            
        print(f'Training loss: {train_loss/len(train_dataloader.dataset):.2f}')
        print(f'Training accuracy: {100*train_accuracy/len(train_dataloader.dataset):.2f}')
        
        if valid_dataloader is not None:
            print(f'Valid loss: {valid_loss/len(valid_dataloader.dataset):.2f}')
            print(f'Valid accuracy: {100*valid_accuracy/len(valid_dataloader.dataset):.2f}')
        
        if epoch%checkpoint_epochs==0:
            torch.save({
                'epoch': epoch,
                'state_dict': net.state_dict(),
                'optimizer': optimizer.state_dict(),
            }, './checkpoint.pth.tar')
        
        print()
        
        if (100*train_accuracy/len(train_dataloader.dataset))>97:
            break
    end = time.time()
    print(f'Total training time: {end-start:.1f} seconds')
    return net


def train_eval(model,save_path):
    train_img_aug_data = SDSSDataset("./balanced_train.csv","./balanced_images")
    test_img_aug_data = SDSSDataset("./balanced_test.csv","./balanced_images")
    train_loader = data.DataLoader(train_img_aug_data, batch_size=64, shuffle=True, 
                                   drop_last=True, pin_memory=True, num_workers=NUM_WORKERS)
    test_loader = data.DataLoader(test_img_data, batch_size=64, shuffle=False, 
                                  drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    lr, weight_decay, epochs = 1e-5, 5e-4, 20

    net = get_net(model).to(device)

    # Standard CrossEntropy Loss for multi-class classification problems
    criterion = torch.nn.CrossEntropyLoss()

    # params_1x are the parameters of the network body, i.e., of all layers except the FC layers
    params_1x = [param for name, param in net.named_parameters() if 'fc' not in str(name)]
    optimizer = torch.optim.Adam([{'params':params_1x}, {'params': net.fc.parameters(), 'lr': lr*10}], lr=lr, weight_decay=weight_decay)

    net = train(net, train_loader, test_loader, criterion, optimizer, None, epochs, device)
    

    net.eval()
    labels = []
    testlabels = []
    test_loader = data.DataLoader(test_img_aug_data, batch_size=1, shuffle=False, 
                                      drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
    for inputs,labels1 in tqdm(test_loader):
        inputs = inputs.to(device)
        testlabels.append(labels1.detach().cpu().numpy()[0])
        labels1 = torch.tensor(labels1).to(device)
        outputs = net(inputs)
        _, predictions = torch.max(outputs, 1)
        predictions = predictions.to("cpu")
        labels.extend(predictions.numpy())
    torch.save(net.state_dict(),save_path)
    return classification_report(testlabels,labels,digits=6).split("\n")
    

In [32]:
train_eval(torchvision.models.resnet18(pretrained=True),"./resnet18_finetune.pth")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

   class           image
0      1  image_3468.jpg
1      1  image_7907.jpg
   class           image
0      2  image_0106.jpg
1      0  image_0440.jpg
Training for 20 epochs on cuda
Epoch 1/20
Training loss: 0.82
Training accuracy: 75.59
Valid loss: 0.39
Valid accuracy: 87.65

Epoch 2/20
Training loss: 0.25
Training accuracy: 91.56
Valid loss: 0.33
Valid accuracy: 89.35

Epoch 3/20
Training loss: 0.17
Training accuracy: 94.51
Valid loss: 0.31
Valid accuracy: 89.90

Epoch 4/20
Training loss: 0.12
Training accuracy: 96.58
Valid loss: 0.31
Valid accuracy: 90.55

Epoch 5/20
Training loss: 0.09
Training accuracy: 97.70
Valid loss: 0.31
Valid accuracy: 91.05

Total training time: 62.4 seconds


  0%|          | 0/2000 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:105: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


['              precision    recall  f1-score   support',
 '',
 '           0   0.962302  0.970000  0.966135      1000',
 '           1   0.897887  0.921687  0.909631       830',
 '           2   0.614286  0.505882  0.554839       170',
 '',
 '    accuracy                       0.910500      2000',
 '   macro avg   0.824825  0.799190  0.810202      2000',
 'weighted avg   0.905988  0.910500  0.907726      2000',
 '']

In [33]:
train_eval(torchvision.models.resnet34(pretrained=True),"./resnet34_finetune.pth")

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

   class           image
0      1  image_3468.jpg
1      1  image_7907.jpg
   class           image
0      2  image_0106.jpg
1      0  image_0440.jpg
Training for 20 epochs on cuda
Epoch 1/20
Training loss: 0.85
Training accuracy: 74.29
Valid loss: 0.37
Valid accuracy: 88.00

Epoch 2/20
Training loss: 0.24
Training accuracy: 92.19
Valid loss: 0.34
Valid accuracy: 89.20

Epoch 3/20
Training loss: 0.15
Training accuracy: 95.21
Valid loss: 0.32
Valid accuracy: 90.20

Epoch 4/20
Training loss: 0.09
Training accuracy: 97.24
Valid loss: 0.33
Valid accuracy: 89.95

Total training time: 57.7 seconds


  0%|          | 0/2000 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:105: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


['              precision    recall  f1-score   support',
 '',
 '           0   0.950389  0.977000  0.963511      1000',
 '           1   0.896719  0.889157  0.892922       830',
 '           2   0.563758  0.494118  0.526646       170',
 '',
 '    accuracy                       0.899500      2000',
 '   macro avg   0.803622  0.786758  0.794360      2000',
 'weighted avg   0.895253  0.899500  0.897083      2000',
 '']

In [34]:
train_eval(torchvision.models.resnet50(pretrained=True),"./resnet50_finetune.pth")

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

   class           image
0      1  image_3468.jpg
1      1  image_7907.jpg
   class           image
0      2  image_0106.jpg
1      0  image_0440.jpg
Training for 20 epochs on cuda
Epoch 1/20
Training loss: 0.56
Training accuracy: 80.69
Valid loss: 0.31
Valid accuracy: 89.80

Epoch 2/20
Training loss: 0.20
Training accuracy: 93.20
Valid loss: 0.26
Valid accuracy: 91.65

Epoch 3/20
Training loss: 0.12
Training accuracy: 96.15
Valid loss: 0.26
Valid accuracy: 92.15

Epoch 4/20
Training loss: 0.07
Training accuracy: 98.19
Valid loss: 0.27
Valid accuracy: 92.25

Total training time: 70.1 seconds


  0%|          | 0/2000 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:105: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


['              precision    recall  f1-score   support',
 '',
 '           0   0.962339  0.971000  0.966650      1000',
 '           1   0.917973  0.916867  0.917420       830',
 '           2   0.697531  0.664706  0.680723       170',
 '',
 '    accuracy                       0.922500      2000',
 '   macro avg   0.859281  0.850858  0.854931      2000',
 'weighted avg   0.921419  0.922500  0.921916      2000',
 '']

In [35]:
train_eval(torchvision.models.resnet101(pretrained=True),"./resnet101_finetune.pth")

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

   class           image
0      1  image_3468.jpg
1      1  image_7907.jpg
   class           image
0      2  image_0106.jpg
1      0  image_0440.jpg
Training for 20 epochs on cuda
Epoch 1/20
Training loss: 0.62
Training accuracy: 79.35
Valid loss: 0.46
Valid accuracy: 88.55

Epoch 2/20
Training loss: 0.17
Training accuracy: 94.28
Valid loss: 0.33
Valid accuracy: 90.95

Epoch 3/20
Training loss: 0.09
Training accuracy: 97.29
Valid loss: 0.37
Valid accuracy: 90.95

Total training time: 77.3 seconds


  0%|          | 0/2000 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:105: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


['              precision    recall  f1-score   support',
 '',
 '           0   0.961386  0.971000  0.966169      1000',
 '           1   0.903846  0.906024  0.904934       830',
 '           2   0.607595  0.564706  0.585366       170',
 '',
 '    accuracy                       0.909500      2000',
 '   macro avg   0.824276  0.813910  0.818823      2000',
 'weighted avg   0.907435  0.909500  0.908388      2000',
 '']